# Worksheet 08

Name:  Maha Alali
UID: U84088912

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [15]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.019624166843549, 4.115990558985977, 4.825878514272246, 4.760529082611539, 5.363528246059516, 5.910251904347891, 4.382911347804442, 4.211740105989784, 5.1230555524470835, 4.560242167826471]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [22]:
s2 = np.random.normal( 8, 1, 10).tolist()
print(s2)

[6.507700723992155, 8.600907024462503, 9.137209201958125, 6.704264294032779, 6.652229286458011, 8.453037096711604, 7.192287193869117, 7.592317977148471, 6.766002420631127, 8.15578552801471]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [23]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.760529082611539, 5.363528246059516, 8.15578552801471, 6.766002420631127, 7.592317977148471, 5.910251904347891, 7.192287193869117, 4.382911347804442, 8.453037096711604, 6.652229286458011]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The prior pr of being s1 = 0.5
The prior pr of being s2 = 0.5
The mean and var of the weights of s1 - s1(mean = 5, var = 1^2)
the mean and var of the weights of s2 - s2(mean = 8, var = 1^2)

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [48]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2)/(len(s1)+len(s2))]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.15578552801471, 6.766002420631127, 7.592317977148471, 7.192287193869117, 8.453037096711604, 6.652229286458011]
[4.760529082611539, 5.363528246059516, 5.910251904347891, 4.382911347804442]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 7.468609917138839,  mean_2 = 5.1043051452058466
var_1 = 0.4488499586450696,  var_2 = 0.3388344466824552


/Users/maha/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The means are close to their corresponding true values whereas the variences are slightly futher away from the true values.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [49]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]
    
    # pdf_i[0] =  p(p | s_0)

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

print("probs")
probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.760529082611539
probability of observing that point if it came from cluster 0 =  1.1073749067753876e-08
probability of observing that point if it came from cluster 1 =  0.7037124952308929
point =  5.363528246059516
probability of observing that point if it came from cluster 0 =  1.4877441123428724e-05
probability of observing that point if it came from cluster 1 =  0.8786745631991582
point =  8.15578552801471
probability of observing that point if it came from cluster 0 =  0.27532425374223923
probability of observing that point if it came from cluster 1 =  2.8791174598366175e-18
point =  6.766002420631127
probability of observing that point if it came from cluster 0 =  0.2610527003581901
probability of observing that point if it came from cluster 1 =  7.0528968248345165e-06
point =  7.592317977148471
probability of observing that point if it came from cluster 0 =  0.8556852445507028
probability of observing that point if it came from cluster 1 =  2.306088961662227e-12
point 


- Point 1: 4.760529082611539 has a higher probability of being in cluster S_2 which alligns with the truth (orginally assigned to cluster with mean = 5). 

- Point 2: 5.363528246059516 has a higher probabilty of being in cluster S_2 which alligns with the truth (orginally assigned to cluster with mean = 5).

- Point 3: 8.15578552801471 has a higher probablilty of being in cluster S_1 which aligns with the truth (orginally assigned to cluster with mean = 8).

- Point 4: 6.766002420631127 has a higher probablilty of being in cluster S_1 which aligns with the truth (orginally assigned to cluster with mean = 8).

- Point 5: 7.592317977148471 has a higher probablilty of being in cluster S_1 which aligns with the truth (orginally assigned to cluster with mean = 8).

- Point 6: 5.910251904347891 has a higher probablilty of being in cluster S_2 which aligns with the truth (orginally assigned to cluster with mean = 5).

- Point 7: 7.192287193869117 has a higher probablilty of being in cluster S_1 which aligns with the truth (orginally assigned to cluster with mean = 8).

- Point 8: 4.382911347804442 has a higher probablilty of being in cluster S_2 which aligns with the truth (orginally assigned to cluster with mean = 5).

- Point 9: 8.453037096711604 has a higher probablilty of being in cluster S_1 which aligns with the truth (orginally assigned to cluster with mean = 8).

- Point 10: 6.652229286458011 has a higher probablilty of being in cluster S_1 which aligns with the truth (orginally assigned to cluster with mean = 8).




f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [52]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 7.45722599338077,  mean_2 = 5.095359777448674
var_1 = 0.463208141095069,  var_2 = 0.3353663142533721


The values of the mean are getting to the orginal values of the mean we got from Kmeans.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [53]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]
    
    # pdf_i[0] =  p(p | s_0)

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

print("probs")
probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  4.760529082611539
probability of observing that point if it came from cluster 0 =  3.7612336242652646e-08
probability of observing that point if it came from cluster 1 =  0.72266405292831
point =  5.363528246059516
probability of observing that point if it came from cluster 0 =  3.153129573531329e-05
probability of observing that point if it came from cluster 1 =  0.8640630426215493
point =  8.15578552801471
probability of observing that point if it came from cluster 0 =  0.27622835569694776
probability of observing that point if it came from cluster 1 =  9.818165656569233e-19
point =  6.766002420631127
probability of observing that point if it came from cluster 0 =  0.2828697509787654
probability of observing that point if it came from cluster 1 =  4.860808486382175e-06
point =  7.592317977148471
probability of observing that point if it came from cluster 0 =  0.8253993609624174
probability of observing that point if it came from cluster 1 =  1.0910482434591354e-12
point =  5

No major difference (none of the updated probabilites indicated a need to switch which cluster they should be assigned to). 

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [59]:
hard_assignment = []
probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))

    if p[1] > p[2]:
        assignment = "S_1"
    else:
        assignment = "S_2" 

    hard_assignment += [(p[0], assignment)]
    print("hard assignment: "+ assignment)
    print()

print(hard_assignment)

4.760529082611539
Probability of coming from S_1 = 7.80701623814877e-08
Probability of coming from S_2 = 0.9999999219298377
hard assignment: S_2

5.363528246059516
Probability of coming from S_1 = 5.473484280580563e-05
Probability of coming from S_2 = 0.9999452651571942
hard assignment: S_2

8.15578552801471
Probability of coming from S_1 = 1.0
Probability of coming from S_2 = 2.3695770676878704e-18
hard assignment: S_1

6.766002420631127
Probability of coming from S_1 = 0.999988544190898
Probability of coming from S_2 = 1.145580910193767e-05
hard assignment: S_1

7.592317977148471
Probability of coming from S_1 = 0.9999999999991187
Probability of coming from S_2 = 8.812285664852866e-13
hard assignment: S_1

5.910251904347891
Probability of coming from S_1 = 0.07295437671838051
Probability of coming from S_2 = 0.9270456232816195
hard assignment: S_2

7.192287193869117
Probability of coming from S_1 = 0.9999999964868114
Probability of coming from S_2 = 3.513188597572497e-09
hard assignm